# Density Based Clustering

<div class="alert alert-block alert-success">
<b>Goals:</b> 

* Learn technical aspects applying DBSCAN.
* Compare to k-means clustering and explore the difference in approach.
* This is a rather technical notebook, ignoring cluster interpretation etc.
</div>

<div class="alert alert-block alert-info">
<b>Content:</b> In this notebook, we 
    
* demonstrate the usage of density based clustering
* try DBSCAN and adjust its parameters to see their influence
* try OPTICS to produce a generally applicable visualization that allows us to determine clustering parameters
</div>

* Experiments are based on the 
    * [sklearn demo](https://scikit-learn.org/stable/auto_examples/cluster/plot_optics.html#sphx-glr-auto-examples-cluster-plot-optics-py) by  Authors: Shane Grigsby <refuge@rocktalus.com> and Adrin Jalali <adrin.jalali@gmail.com> under License BSD 3 clause, and 
    * [k-means demo](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html) by author: Author: Phil Roth <mr.phil.roth@gmail.com> under License BSD 3 clause
* Parts of the code have been modified and extended to fit the context of the lecture



In [ ]:
# Original Licence Remark (OPTICS)
# Authors: Shane Grigsby <refuge@rocktalus.com>
#          Adrin Jalali <adrin.jalali@gmail.com>
# License: BSD 3 clause

# Original Licence Remark (k-Means)
# Author: Phil Roth <mr.phil.roth@gmail.com>
# License: BSD 3 clause

## Part I: DBSCAN

### Generate sample data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
n_points_per_cluster = 250

# create 6 clusters with varying density
C1 = [-5, -2] + 0.8 * np.random.randn(n_points_per_cluster, 2)
C2 = [4, -1] + 0.1 * np.random.randn(n_points_per_cluster, 2)
C3 = [1, -2] + 0.2 * np.random.randn(n_points_per_cluster, 2)
C4 = [-2, 3] + 0.3 * np.random.randn(n_points_per_cluster, 2)
C5 = [3, -2] + 1.6 * np.random.randn(n_points_per_cluster, 2)
C6 = [5, 6] + 2 * np.random.randn(n_points_per_cluster, 2)
X = np.vstack((C1, C2, C3, C4, C5, C6))

In [ ]:
plt.scatter(X[:,0], X[:, 1], alpha=0.1)

In [ ]:
from sklearn.cluster import DBSCAN
dbscan=DBSCAN(eps=1, min_samples=10, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=3)
dbscan_clusters=dbscan.fit_predict(X)
print(np.unique(dbscan_clusters)) # -1 is the marker for noise instances

In [ ]:
colors=['darkorange', 'darkmagenta', 'dodgerblue', 'lightgreen']

In [ ]:
def plot_clusters(X, clusters):
    for cluster in np.unique(clusters):
        if (cluster!=-1):
            X_cluster=X[clusters==cluster, :]
            plt.scatter(X_cluster[:,0], X_cluster[:,1], c=colors[cluster])
    X_noise=X[clusters==-1, :]
    plt.scatter(X_noise[:,0], X_noise[:,1], c='black')


In [ ]:
plot_clusters(X,dbscan_clusters)

In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans(4, n_init=10)
kmeans_clusters=kmeans.fit_predict(X)
plot_clusters(X,kmeans_clusters)

### Comparison
* The clusters of k-Means and DBSCAN are roughly similar.
* However, with k-Means, all instances are assigned to clusters, whereas DBSCAN separates data from noise.
* For k-Means we had to specify $k$, for DBSCAN, we had to specify $\epsilon$ and $\text{MinPts}$.

<div class="alert alert-block alert-info">
<b>Observation:</b>    
For k-Means we specify the number of clusters, for DBSCAN, we specify properties of the clusters.
</div>

### Generate Anisotrophicly Shaped Clusters
This was one of the examples, that k-means returned results that did not fit the intuition.

In [ ]:
from sklearn.datasets import make_blobs
n_samples = 1500
random_state = 170
X_plain, y = make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X_plain, transformation)
plt.scatter(X_aniso[:,0], X_aniso[:,1])

In [ ]:
kmeans=KMeans(3, n_init=10)
kmeans_clusters=kmeans.fit_predict(X_aniso)
plot_clusters(X_aniso, kmeans_clusters)

In [ ]:
dbscan=DBSCAN(eps=.4, min_samples=10, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=1)
dbscan_clusters=dbscan.fit_predict(X_aniso)
plot_clusters(X_aniso, dbscan_clusters)

### The Influence of Parameters

In [ ]:
epsilons=(1, 0.4, 0.2)
min_ptss=(1, 10, 20, 100)

point_size=2

epss, ptss = np.meshgrid(epsilons, min_ptss)
_, axes = plt.subplots(nrows=len(min_ptss), ncols=len(epsilons), figsize=(18, 10))
for ax, eps, min_pts in zip(axes.reshape(-1), epss.reshape(-1) , ptss.reshape(-1)):
    ax.set_axis_off()
    dbscan=DBSCAN(eps=eps, min_samples=min_pts, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=1)
    dbscan.fit_predict(X_aniso)
    y_pred=dbscan.labels_
    if (len(np.unique(y_pred))>len(colors)): 
        for i in np.unique(y_pred):
            ax.scatter(X_aniso[y_pred==i, 0], X_aniso[y_pred==i, 1], s=point_size)
        ax.scatter(X_aniso[y_pred==-1, 0], X_aniso[y_pred==-1, 1], c='black', s=point_size)

        
    else:
        for i in np.unique(y_pred):
            ax.scatter(X_aniso[y_pred==i, 0], X_aniso[y_pred==i, 1], c=colors[i], s=point_size)                
    ax.set_title(r'$\epsilon=$%.2f, minPts=%i' % (eps, min_pts))

<div class="alert alert-block alert-warning">
    How do we choose parameters without looking at data plots?
</div>

## Part II: OPTICS
Let's go back to the original example:

In [ ]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec

In [ ]:
plt.scatter(X[:,0], X[:, 1], alpha=0.1)

In [ ]:
optics = OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05)
optics.fit(X) # learn the optics diagram

Let's create three different clusterings:
* one with OPTICS's $\xi$-method
* two by extracting DBSCAN clusterings from the OPTICS diagram using vertical $\epsilon$-cuts

In [ ]:
labels_xi=optics.labels_

labels_050 = cluster_optics_dbscan(
    reachability=optics.reachability_,
    core_distances=optics.core_distances_,
    ordering=optics.ordering_,
    eps=0.5,
)
labels_200 = cluster_optics_dbscan(
    reachability=optics.reachability_,
    core_distances=optics.core_distances_,
    ordering=optics.ordering_,
    eps=2,
)

In [ ]:
np.unique(labels_xi), np.unique(labels_050), np.unique(labels_200)

### Visualization and Relationship of the Clusterings
* We visualize the OPTICS diagram and the clusters.
* Note, that this visualization is __independent__ of the number of features in the dataset, thus always available!

In [ ]:
space = np.arange(len(X))
reachability = optics.reachability_[optics.ordering_]

plt.figure(figsize=(10, 3.5))
plt.scatter(space, reachability, alpha=0.3)

In [ ]:
labels = labels_xi[optics.ordering_]

plt.figure(figsize=(10, 7))
G = gridspec.GridSpec(2, 3)
ax1 = plt.subplot(G[0, :])
ax2 = plt.subplot(G[1, 0])
ax3 = plt.subplot(G[1, 1])
ax4 = plt.subplot(G[1, 2])

# Reachability plot
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    ax1.plot(Xk, Rk, color, alpha=0.3)
ax1.plot(space[labels == -1], reachability[labels == -1], "k.", alpha=0.3)
ax1.plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
ax1.plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax1.set_ylabel("Reachability distance")
ax1.set_title("Reachability Diagram")

# OPTICS
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = X[optics.labels_ == klass]
    ax2.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
ax2.plot(X[labels_xi == -1, 0], X[labels_xi == -1, 1], "k+", alpha=0.1)
ax2.set_title(r'OPTICS $\xi$ Method')

# DBSCAN at 0.5
#colors = ["g", "greenyellow", "olive", "r", "b", "c"]
colors = ["g", "r", "b", "c"]
for klass, color in zip(range(0, 6), colors):
    Xk = X[labels_050 == klass]
    ax3.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3, marker=".")
ax3.plot(X[labels_050 == -1, 0], X[labels_050 == -1, 1], "k+", alpha=0.1)
ax3.set_title("Clustering at 0.5 epsilon cut\nDBSCAN")

# DBSCAN at 2.
colors = ["g.", "m.", "y.", "c."]
for klass, color in zip(range(0, 4), colors):
    Xk = X[labels_200 == klass]
    ax4.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
ax4.plot(X[labels_200 == -1, 0], X[labels_200 == -1, 1], "k+", alpha=0.1)
ax4.set_title("Clustering at 2.0 epsilon cut\nDBSCAN")

plt.tight_layout()
plt.show()

<div class="alert alert-block alert-info">
<b>Take-Aways:</b>    

* For density-based clustering, parameters $\epsilon$ and MinPts specify what density means, thus the properties of clustering
* Density-based clustering detects and excludes noise
* It is non-trivial to choose "good" parameters. Often a visual analysis like above (12 visualizations of the data) is impossible due to the number of dimensions.
* OPTICS can be used to visually aid with the parameter choice.
* OPTICS produces a visualization that is applicable with every dataset.
* DBSCAN clusterings with different $\epsilon$ can be "harvested" from the OPTICS result.
</div>

<div class="alert alert-block alert-success">
<b>Play with:</b> 

* Play with different parameters, even extremely high and extremely low choices.
* Create a three dimensional dataset. What happens with the visual exploration?
* Use the IRIS dataset and cluster it using DBSCAN or OPTICS
</div>